**TRANSFORMACIONES - files/World_Happiness_Report.csv**

*SIN IMPUTAR - solo nuevas columnas de % y binario (yes/no)*

__________

**Where is this data set from?**

- The World Happiness Report is an annual publication of the United Nations Sustainable Development Solutions Network. This dataset is a subset of the larger report, which includes data from various sources such as the Gallup World Poll and other national surveys. The data was extracted from the World Happiness Report and made available for public use. However, the original data was collected by various researchers and organizations as part of their ongoing efforts to measure and understand happiness and well-being around the world.

    We use observed data on the six variables and estimates of their associations with life evaluations to explain the variation across countries. They include GDP per capita, social support, healthy life expectancy, freedom, generosity, and corruption. Our happiness rankings are not based on any index of these six factors – the scores are instead based on individuals’ own assessments of their lives, in particular, their answers to the single-item Cantril ladder life-evaluation question, much as epidemiologists estimate the extent to which life expectancy is affected by factors such as smoking, exercise, and diet

Detailed information about each of the Predictors:

1. **Log GDP per capita** is in terms of Purchasing Power Parity (PPP) adjusted to a constant 2017 international dollars, taken from the World Development Indicators (WDI) by the World Bank (version 17, metadata last updated on January 22, 2023). See Statistical Appendix 1 for more details. GDP data for 2022 are not yet available, so we extend the GDP time series from 2021 to 2022 using country-specific forecasts of real GDP growth from the OECD Economic Outlook No. 112 (November 2022) or, if missing, from the World Bank’s Global Economic Prospects (last updated: January 10, 2023), after adjustment for population growth. The equation uses the natural log of GDP per capita, as this form fits the data significantly better than GDP per capita.

2. The time series for **Healthy life expectancy at birth** is constructed based on data from the World Health Organization (WHO) Global Health Observatory data repository, with data available for 2005, 2010, 2015, 2016, and 2019. To match this report’s sample period (2005-2022), interpolation and extrapolation are used. See Statistical Appendix 1 for more details.

3. **Social support** - *Conversion: % y yes/no*

    **Social support** (0-1) is the national average of the binary responses (0=no, 1=yes) to the Gallup World Poll (GWP) question “If you were in trouble, do you have relatives or friends you can count on to help you whenever you need them, or not?”

4.  **Freedom to make life choices** - *Conversion: % y yes/no* 

    **Freedom to make life choices** (0-1) is the national average of binary responses to the GWP question “Are you satisfied or dissatisfied with your freedom to choose what you do with your life?”

5. **Generosity** is the residual of regressing the national average of GWP responses to the donation question “Have you donated money to a charity in the past month?” on log GDP per capita.

6.  **Perceptions of corruption** - *Conversion: % y yes/no*  

    **Perceptions of corruption** (0-1) are the average of binary answers to two GWP questions: “Is corruption widespread throughout the government or not?” and “Is corruption widespread within businesses or not?” Where data for government corruption are missing, the perception of business corruption is used as the overall corruption perception measure.

7. **Positive affect** - *Conversion: % y yes/no* 
**Positive affect** is defined as the average of previous-day effects measures for laughter, enjoyment, and interest. The inclusion of interest (first added for World Happiness Report 2022), gives us three components in each of positive and negative affect, and slightly improves the equation fit in column 4. The general form for the affect questions is: Did you experience the following feelings during a lot of the day yesterday?

8. **Negative affect** - *Conversion: % y yes/no* 
**Negative affect** is defined as the average of previous-day effects measures for worry, sadness, and anger.

9. **Life ladder**: Life evaluations from the Gallup World Poll provide the basis for the annual happiness rankings. They are based on answers to the main life evaluation question. The Cantril ladder asks respondents to think of a ladder, with the **best possible life for them being a 10 and the worst possible life being a 0**. They are then asked to rate their own current lives **on a 0 to 10 scale**. The rankings are from nationally representative samples over three years.

10. **Confidence in National Government**: The "Confidence in National Government" variable in the World Happiness Report is calculated based on the following question asked in the Gallup World Poll:

    "Do you have confidence in the national government?"

    Respondents are given the following options to choose from:

    - "Yes, always"
    - "Yes, sometimes"
    - "No, rarely"
    - "No, never"
    - "Don't know"

    **The variable is calculated as the percentage of respondents who answer "Yes, always" or "Yes, sometimes" to this question.**

    This variable is one of several social factors that are included in the calculation of the World Happiness Report's overall happiness score for each country. The report combines data on social factors such as income, social support, life expectancy, freedom to make life choices, generosity, and perceptions of corruption to arrive at a comprehensive measure of happiness.

In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# ------------------------------------------------------------------------------
from scipy.stats import shapiro, kstest

# Scripts de soporte
# -------------------------------------------------
from src.tolookandcompare import to_doc_info, to_doc_headtail, transform_info, transform_headtail

from src import soporte_eda as sp_eda
from src.soporte_eda import resumen_df

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames


**Leer/subir el .csv**

**TRANSFORMACION de `Country Name` al subir el .csv**

In [2]:
# Se edito src --> tolookandcompare_v2.py que estaba rompiendo 'Country Name'
df = pd.read_csv('files/World_Happiness_Report.csv') 

df.sample(5)

,Country Name,Regional Indicator,Year,Life Ladder,Log GDP Per Capita,Social Support,Healthy Life Expectancy At Birth,Freedom To Make Life Choices,Generosity,Perceptions Of Corruption,Positive Affect,Negative Affect,Confidence In National Government
265,Bulgaria,Central and Eastern Europe,2015,4.865401,9.903078,0.907517,65.900002,0.636818,-0.204797,0.941280,0.555952,0.214224,0.249677
369,Chile,Latin America and Caribbean,2011,6.526335,10.013405,0.819079,68.680000,0.700734,0.105664,0.752756,0.757601,0.316876,0.322867
1951,Togo,Sub-Saharan Africa,2008,2.807855,7.312004,0.291334,51.119999,0.286814,-0.074570,0.931986,0.397701,0.378715,0.170061
470,Croatia,Central and Eastern Europe,2018,5.536271,10.247964,0.909807,68.425003,0.690856,-0.156302,0.925408,0.512390,0.290376,0.248181
1895,Taiwan Province of China,East Asia,2018,6.467005,10.780802,0.896459,NaN,0.741033,-0.176948,0.735971,0.745919,0.092696,0.311723


In [3]:
df.shape

(2199, 13)

In [4]:
df['Country Name'].dtype

dtype('O')

In [5]:
# NULOS - 0
df['Country Name'].isna().sum()

np.int64(0)

In [6]:
df['Country Name'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Argentina',
       'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain',
       'Bangladesh', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon',
       'Canada', 'Central African Republic', 'Chad', 'Chile', 'China',
       'Colombia', 'Comoros', 'Congo (Brazzaville)', 'Congo (Kinshasa)',
       'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Denmark',
       'Djibouti', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvador', 'Estonia', 'Eswatini', 'Ethiopia', 'Finland',
       'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana',
       'Greece', 'Guatemala', 'Guinea', 'Guyana', 'Haiti', 'Honduras',
       'Hong Kong S.A.R. of China', 'Hungary', 'Iceland', 'India',
       'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy',
       'Ivory Coast', 'Jamaica', 'Japan', 

In [7]:
df['Country Name'].value_counts()

Country Name
Argentina     17
Costa Rica    17
Brazil        17
Bolivia       17
Bangladesh    17
              ..
Cuba           1
Maldives       1
Guyana         1
Oman           1
Suriname       1
Name: count, Length: 165, dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2199 entries, 0 to 2198
Data columns (total 13 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Country Name                       2199 non-null   object 
 1   Regional Indicator                 2087 non-null   object 
 2   Year                               2199 non-null   int64  
 3   Life Ladder                        2199 non-null   float64
 4   Log GDP Per Capita                 2179 non-null   float64
 5   Social Support                     2186 non-null   float64
 6   Healthy Life Expectancy At Birth   2145 non-null   float64
 7   Freedom To Make Life Choices       2166 non-null   float64
 8   Generosity                         2126 non-null   float64
 9   Perceptions Of Corruption          2083 non-null   float64
 10  Positive Affect                    2175 non-null   float64
 11  Negative Affect                    2183 non-null   float

_________________________

📊 Transformación del PIB:

Se ha transformado/deslogaritmado la variable `Log GDP Per Capita` para obtener un PIB per cápita más interpretable.
Además, se ha creado una métrica sencilla de felicidad relativa al nivel de riqueza.

El código es robusto e idempotente, preparado para valores nulos y ejecuciones repetidas, y los resultados se utilizan posteriormente en Power BI para facilitar el análisis y la visualización.

Columnas Nuevas: 
- `GDP` - PIB per capita *real* (sueldo anual)
- `GDP_k`- GDP dividido or 1000 (para las etiquetas 1k = 1000 / 65,000 = 65k - *mas limpio*)
- `Happiness_per_GDP` - Es una métrica relativa, no absoluta.

**Happiness_per_GDP** - Es una métrica relativa, no absoluta.

Fórmula simplificada:

felicidad ÷ riqueza

🧠 Sirve para responder:

“¿Quién es más feliz para el dinero que tiene?”

“¿Quién convierte mejor riqueza en bienestar?”

✔️ Ideal para rankings y comparaciones
❌ No usar como valor económico real

In [9]:
# 1. GDP desde log (si hay NaN, se queda NaN)
df["GDP"] = np.exp(df["Log GDP Per Capita"])

# 2. GDP en miles (evita divisiones raras)
df["GDP_k"] = df["GDP"] / 1000

# 3. Felicidad relativa al GDP
# Solo se calcula si GDP_k > 0 y Life Ladder no es nulo
df["Happiness_per_GDP"] = np.where(
    (df["GDP_k"] > 0) & (df["Life Ladder"].notna()),
    df["Life Ladder"] / df["GDP_k"],
    np.nan
)

In [10]:
df.columns

Index(['Country Name', 'Regional Indicator', 'Year', 'Life Ladder',
       'Log GDP Per Capita', 'Social Support',
       'Healthy Life Expectancy At Birth', 'Freedom To Make Life Choices',
       'Generosity', 'Perceptions Of Corruption', 'Positive Affect',
       'Negative Affect', 'Confidence In National Government', 'GDP', 'GDP_k',
       'Happiness_per_GDP'],
      dtype='object')

_________________________

**TRANSFORMACION de las Columnas (5):**

- `Social Support`
- `Freedom To Make Life Choices`
- `Perceptions Of Corruption`
- `Positive Affect`
- `Negative Affect`

Columnas Nuevas: 
- valor numerico decimal en % *(% de si)* 
- valor categorico en binario *(No = hasta .5, Yes = .5+)*, NaN categorico a *"Data not available"*
- sin imputar nulos

In [11]:
# Lista de columnas originales que queremos transformar
cols = [
    "Social Support",
    "Freedom To Make Life Choices",
    "Perceptions Of Corruption",
    "Positive Affect",
    "Negative Affect"
]

for col in cols:
    print(f"\n🔎 Procesando columna: {col}")

    # 1. Contar nulos antes de transformar
    n_nulos = df[col].isna().sum()
    print(f"   ➤ Nulos encontrados: {n_nulos}")

    # 2. Convertir el nombre a Title Case para consistencia en las columnas nuevas
    col_title = col.title()

    # 3. Crear columna en formato porcentaje
    df[f"{col_title} (%)"] = (df[col] * 100).round(1)
    print(f"   ✔ Columna creada: '{col_title} (%)'")

    # 4. Crear columna categórica Yes/No
    df[f"{col_title} (Yes/No)"] = df[col].apply(
        lambda x: "Yes" if pd.notnull(x) and x >= 0.5 else
                  "No" if pd.notnull(x) else
                  "Data not available"
    )
    print(f"   ✔ Columna creada: '{col_title} (Yes/No)'")

    # 5. Mostrar ejemplo rápido de los primeros valores transformados
    print(df[[col, f"{col_title} (%)", f"{col_title} (Yes/No)"]].head(3))




🔎 Procesando columna: Social Support
   ➤ Nulos encontrados: 13
   ✔ Columna creada: 'Social Support (%)'
   ✔ Columna creada: 'Social Support (Yes/No)'
   Social Support  Social Support (%) Social Support (Yes/No)
0        0.450662                45.1                      No
1        0.552308                55.2                     Yes
2        0.539075                53.9                     Yes

🔎 Procesando columna: Freedom To Make Life Choices
   ➤ Nulos encontrados: 33
   ✔ Columna creada: 'Freedom To Make Life Choices (%)'
   ✔ Columna creada: 'Freedom To Make Life Choices (Yes/No)'
   Freedom To Make Life Choices  Freedom To Make Life Choices (%)  \
0                      0.718114                              71.8   
1                      0.678896                              67.9   
2                      0.600127                              60.0   

  Freedom To Make Life Choices (Yes/No)  
0                                   Yes  
1                                   Yes  

*NaNs % se mantienen porque Tableau/Power BI pueden graficarla sin errores.*

In [12]:
df.sample(5)

,Country Name,Regional Indicator,Year,Life Ladder,Log GDP Per Capita,Social Support,Healthy Life Expectancy At Birth,Freedom To Make Life Choices,Generosity,Perceptions Of Corruption,Positive Affect,Negative Affect,Confidence In National Government,GDP,GDP_k,Happiness_per_GDP,Social Support (%),Social Support (Yes/No),Freedom To Make Life Choices (%),Freedom To Make Life Choices (Yes/No),Perceptions Of Corruption (%),Perceptions Of Corruption (Yes/No),Positive Affect (%),Positive Affect (Yes/No),Negative Affect (%),Negative Affect (Yes/No)
669,Gambia,Sub-Saharan Africa,2019,5.163627,7.641713,0.693870,57.000000,0.676595,0.412655,0.798108,0.717893,0.400723,0.645316,2083.308775,2.083309,2.478570,69.4,Yes,67.7,Yes,79.8,Yes,71.8,Yes,40.1,No
1404,New Zealand,North America and ANZ,2015,7.418121,10.621822,0.987343,69.900002,0.941784,0.324308,0.185889,0.794508,0.159830,0.620908,41020.300611,41.020301,0.180840,98.7,Yes,94.2,Yes,18.6,No,79.5,Yes,16.0,No
1302,Mongolia,East Asia,2015,4.982720,9.317831,0.905524,59.900002,0.685511,0.169322,0.900218,0.532773,0.207653,0.226755,11134.804686,11.134805,0.447491,90.6,Yes,68.6,Yes,90.0,Yes,53.3,Yes,20.8,No
735,Greece,Western Europe,2020,5.787616,10.207419,0.778537,71.025002,0.564614,-0.244105,0.764325,0.629382,0.321684,0.397470,27103.533884,27.103534,0.213537,77.9,Yes,56.5,Yes,76.4,Yes,62.9,Yes,32.2,No
1478,Norway,Western Europe,2016,7.596332,11.052541,0.959743,71.250000,0.954352,0.127090,0.409666,0.809430,0.209262,0.657646,63104.084688,63.104085,0.120378,96.0,Yes,95.4,Yes,41.0,No,80.9,Yes,20.9,No


In [13]:
df['Country Name'].dtype

dtype('O')

In [14]:
# NULOS - sigue siendo 0
df['Country Name'].isna().sum()

np.int64(0)

____

#### **GUARDAR nuevo .csv "World Happiness Report limpio sin imputar":** 

In [15]:
df.to_csv('files/World_Happiness_Report_limpio_sin_imputar.csv', index=False)

In [16]:
df.head(5)

,Country Name,Regional Indicator,Year,Life Ladder,Log GDP Per Capita,Social Support,Healthy Life Expectancy At Birth,Freedom To Make Life Choices,Generosity,Perceptions Of Corruption,Positive Affect,Negative Affect,Confidence In National Government,GDP,GDP_k,Happiness_per_GDP,Social Support (%),Social Support (Yes/No),Freedom To Make Life Choices (%),Freedom To Make Life Choices (Yes/No),Perceptions Of Corruption (%),Perceptions Of Corruption (Yes/No),Positive Affect (%),Positive Affect (Yes/No),Negative Affect (%),Negative Affect (Yes/No)
0,Afghanistan,South Asia,2008,3.723590,7.350416,0.450662,50.500000,0.718114,0.167652,0.881686,0.414297,0.258195,0.612072,1556.844325,1.556844,2.391755,45.1,No,71.8,Yes,88.2,Yes,41.4,No,25.8,No
1,Afghanistan,South Asia,2009,4.401778,7.508646,0.552308,50.799999,0.678896,0.190809,0.850035,0.481421,0.237092,0.611545,1823.742543,1.823743,2.413596,55.2,Yes,67.9,Yes,85.0,Yes,48.1,No,23.7,No
2,Afghanistan,South Asia,2010,4.758381,7.613900,0.539075,51.099998,0.600127,0.121316,0.706766,0.516907,0.275324,0.299357,2026.164160,2.026164,2.348468,53.9,Yes,60.0,Yes,70.7,Yes,51.7,Yes,27.5,No
3,Afghanistan,South Asia,2011,3.831719,7.581259,0.521104,51.400002,0.495901,0.163571,0.731109,0.479835,0.267175,0.307386,1961.095989,1.961096,1.953866,52.1,Yes,49.6,No,73.1,Yes,48.0,No,26.7,No
4,Afghanistan,South Asia,2012,3.782938,7.660506,0.520637,51.700001,0.530935,0.237588,0.775620,0.613513,0.267919,0.435440,2122.830826,2.122831,1.782025,52.1,Yes,53.1,Yes,77.6,Yes,61.4,Yes,26.8,No
